In [1]:
import re
import html
import pickle
import pandas as pd
import numpy as np
random_state = 20
RS = np.random.RandomState(random_state)

In [2]:
frame_names = ['lime', 'integrated_gradients', 'guided_backprop', 'kernel_shap', 'input_x_gradients', 'deeplift']
frame_names_idx_dict = {frame_names[i]: i for i in range(len(frame_names))}
samples_per_frame = 5

In [3]:
def get_explanations_and_out_qnli(frame_name):
    exps = pd.read_csv(f'{frame_name}_qnli.csv')
    outs = pickle.load(open(f'{frame_name}_out.pkl', 'rb'))
    
    #50 samples per explanation framework
    frame_seed = RS.randint(low=0,high=50)

    random_samples = exps.sample(n=samples_per_frame, random_state=frame_seed)
    rs_idx = random_samples.index

    questions = random_samples['question']
    paragraphs = random_samples['paragraph']    
    
    model_outs = np.array(outs["model_out_list"])[rs_idx]
    outs = ['Entailment' if k > 0.5 else 'Non-entailment' for k in model_outs]

    return questions, paragraphs, outs, rs_idx

In [4]:
def get_explanations_and_out_for_all_frames_qnli(frame_names):
    all_questions, all_paragraphs, all_outs, all_rs_idx = [], [], [], {}
    for frame_name in frame_names:
        questions, paragraphs, outs, rs_idx = get_explanations_and_out_qnli(frame_name)
        all_questions += questions.tolist()
        all_paragraphs += paragraphs.tolist()        
        all_outs += outs
        all_rs_idx[frame_name] = rs_idx.tolist()
    
    return all_questions, all_paragraphs, all_outs, all_rs_idx

In [5]:
qnli_base_usability_file = open('qnli_base.html',mode='r')
qnli_base_html = qnli_base_usability_file.read()
qnli_individual_usability_file = open('qnli_individual.html',mode='r')
qnli_individual_html = qnli_individual_usability_file.read()

def gen_usability_samples(questions, paragraphs, outs):
    all_i_samples = []
    for i, (q,p,o) in enumerate(zip(questions, paragraphs, outs)):
        i_with_example_number = re.sub('\[TO REPLACE EXAMPLE NUMBER\]', str(i), qnli_individual_html)
        i_with_question = re.sub('\[TO REPLACE QUESTION\]', q, i_with_example_number)
        i_with_paragraph = re.sub('\[TO REPLACE PARAGRAPH\]', p, i_with_question)        
        i_with_model_out= re.sub('\[TO REPLACE MODEL OUT\]', o, i_with_paragraph)    
        all_i_samples.append(i_with_model_out)
    
    all_i_samples = '\n\n'.join(all_i_samples)
    return all_i_samples

In [6]:
def gen_usability_test_file(frame_names):
    all_questions, all_paragraphs, all_outs, all_rs_idx = get_explanations_and_out_for_all_frames_qnli(frame_names)
    usability_samples = gen_usability_samples(all_questions, all_paragraphs, all_outs)
    qnli_usability_test_html = re.sub('\[TO REPLACE USABILITY SAMPLES\]', usability_samples, qnli_base_html)
    with open('qnli_usability_test.html', 'w') as f:
        f.write(f'{qnli_usability_test_html}')
#     print(f'{qnli_usability_test_html}')


    with open(f'random_sample_indices.pkl', 'wb') as f:
        pickle.dump(all_rs_idx, f)
    


In [7]:
gen_usability_test_file(frame_names)

In [8]:
with open(f'random_sample_indices.pkl', 'rb') as f:
    thing = pickle.load(f)
    print(thing)
    

{'lime': [39, 26, 22, 31, 29], 'integrated_gradients': [2, 43, 10, 14, 44], 'guided_backprop': [35, 37, 41, 26, 38], 'kernel_shap': [32, 4, 5, 31, 2], 'input_x_gradients': [11, 16, 9, 31, 27], 'deeplift': [2, 43, 10, 14, 44]}
